In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

pd.set_option('display.max_rows', None)

In [44]:
df = pd.read_csv('Flight_delay.csv')

In [ ]:
df.rename(columns={'CRSArrTime': 'ScheduledArrTime'}, inplace=True)

In [ ]:
def create_scheduled_dep_time_col():
    # Convert 'DepTime' to string type
    df['DepTime'] = df['DepTime'].astype(str)

    # Pad 'DepTime' with leading zeros to ensure it has 4 digits
    df['DepTime'] = df['DepTime'].str.zfill(4)

    # Replace '2400' with '0000' in 'DepTime'
    df['DepTime'] = df['DepTime'].replace('2400', '0000')

    # Convert 'DepTime' column to datetime format
    df['DepTime'] = pd.to_datetime(df['DepTime'], format='%H%M')

    # Subtract 'DepDelay' from 'DepTime'
    df['ScheduledDepTime'] = df.apply(lambda row: row['DepTime'] - timedelta(minutes=row['DepDelay']), axis=1)

    # Convert 'DepTime' and 'ScheduledDepTime' back to the original format
    df['DepTime'] = df['DepTime'].dt.strftime('%H%M')
    df['ScheduledDepTime'] = df['ScheduledDepTime'].dt.strftime('%H%M')

In [ ]:
create_scheduled_dep_time_col()

In [ ]:
def upsert_scheduled_elapsed_time():
    df.drop(columns='CRSElapsedTime', inplace=True)
    
    # Convert columns to string type
    df['ScheduledArrTime'] = df['ScheduledArrTime'].astype(str)
    df['ScheduledDepTime'] = df['ScheduledDepTime'].astype(str)

    # Pad columns with leading zeros to ensure it has 4 digits
    df['ScheduledArrTime'] = df['ScheduledArrTime'].str.zfill(4)
    df['ScheduledDepTime'] = df['ScheduledDepTime'].str.zfill(4)
    
    # Replace '2400' with '0000' in columns
    df['ScheduledArrTime'] = df['ScheduledArrTime'].replace('2400', '0000')
    df['ScheduledDepTime'] = df['ScheduledDepTime'].replace('2400', '0000')
    
    # Convert columns to datetime format
    df['ScheduledArrTime'] = pd.to_datetime(df['ScheduledArrTime'], format='%H%M')
    df['ScheduledDepTime'] = pd.to_datetime(df['ScheduledDepTime'], format='%H%M')
    
    # Calculate the scheduled elapsed time and create a new column 'ScheduledElapsedTime'
    df['ScheduledElapsedTime'] = ((df['ScheduledArrTime'] - df['ScheduledDepTime']).dt.total_seconds() / 60).astype(int)
    
    # Convert 'ScheduledArrTime' and 'ScheduledDepTime' back to the original format
    df['ScheduledArrTime'] = df['ScheduledArrTime'].dt.strftime('%H%M')
    df['ScheduledDepTime'] = df['ScheduledDepTime'].dt.strftime('%H%M')

In [ ]:
upsert_scheduled_elapsed_time()